In [1]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.models import load_model
from tensorflow.keras import utils
from wandb.keras import WandbCallback
import wandb

In [2]:
wandb.init(project="resume-read-group-runs", name="group_runs", group="learn_rate")
config = wandb.config

In [3]:
config.dropout = 0.2
config.hidden_layer_size = 128
config.layer_1_size  = 16
config.layer_2_size = 32
config.learn_rate = 0.01

In [4]:
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

img_width=28
img_height=28

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)

# one hot encode outputs
y_train = utils.to_categorical(y_train)
y_test = utils.to_categorical(y_test)
num_classes = y_test.shape[1]

In [5]:
# Build and compile model
def get_trainable_model():
    model = Sequential()
    model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                                input_shape=(img_width, img_height,1)))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(config.layer_2_size, (5, 5), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(config.dropout))
    model.add(Flatten())
    model.add(Dense(config.hidden_layer_size, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(loss='categorical_crossentropy', optimizer="adam", metrics=['accuracy'])
    
    return model

In [6]:
model = get_trainable_model()
model.fit(X_train, y_train,  validation_data=(X_test, y_test), 
          epochs=25, initial_epoch=wandb.run.step,
          callbacks=[WandbCallback(data_type="image", labels=labels, save_model=True)])

Train on 60000 samples, validate on 10000 samples
Epoch 1/25
60000/60000 [==============================] - 13s 217us/sample - loss: 0.5113 - accuracy: 0.8142 - val_loss: 0.3667 - val_accuracy: 0.8659
Epoch 2/25
60000/60000 [==============================] - 11s 181us/sample - loss: 0.3503 - accuracy: 0.8717 - val_loss: 0.3219 - val_accuracy: 0.8815
Epoch 3/25
60000/60000 [==============================] - 11s 180us/sample - loss: 0.3079 - accuracy: 0.8873 - val_loss: 0.3201 - val_accuracy: 0.8827
Epoch 4/25
60000/60000 [==============================] - 11s 179us/sample - loss: 0.2859 - accuracy: 0.8951 - val_loss: 0.2845 - val_accuracy: 0.8960
Epoch 5/25
60000/60000 [==============================] - 11s 179us/sample - loss: 0.2666 - accuracy: 0.9004 - val_loss: 0.2845 - val_accuracy: 0.8947
Epoch 6/25
60000/60000 [==============================] - 11s 179us/sample - loss: 0.2510 - accuracy: 0.9068 - val_loss: 0.2739 - val_accuracy: 0.8996
Epoch 7/25
60000/60000 [====================